# MAP 435 - TP à trous à rendre pour le mardi 9 mai 23h59

Le but de cet exercice est de compléter les fonctions manquantes en remplaçant les parties `# YOUR CODE HERE` par votre code.

**Attention** la correction des notebooks se faisant de manière automatique :
- il faut que vous supprimiez la ligne `raise NotImplementedError()` ;
- les seules cellules que vous devez modifier sont celles comportant la mention `# YOUR CODE HERE` ;
- vous ne devez pas modifier le nom du fichier faute de quoi votre devoir ne sera pas corrigé.

Merci d'inscrire votre nom dans la cellule ci-dessous.

In [ ]:
NOM = "MILLET"

# Algorithme des moindres carrés non-linéaire

Dans ce notebook nous allons utiliser un algorithme qui permet de minimiser des fonctions qui s'écrivent comme des sommes de carrés. Une grande classe de telles fonctionnelles s'obtient à partir de problèmes de recherche de paramètres d'un modèle à partir d'un critère de type moindres carrés. 

Considérons une fonction sous la forme
$$f(x) = \sum_{j=1}^m r_j(x)^2.$$
La matrice Jacobienne associée est
$$ J(x) = \begin{pmatrix}
\frac{\partial r_1}{\partial x_1}& \cdots & \frac{\partial r_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial r_m}{\partial x_1}& \cdots & \frac{\partial r_m}{\partial x_n}
\end{pmatrix}.$$
Il est immédiat de voir que le gradient de $f$ s'écrit sous la forme 
$$ \nabla f(x) = 2(J(x))^T r$$
avec $r = (r_1,...,r_m)^T$. 

La méthode que l'on utilise, appelée méthode de Gauss-Newton, consiste à construire la suite $x_i$ définie par :

\begin{equation}
\left\{
\begin{aligned}
&x_0 \in \mathbb R^N \text{ quelconque} \\
& x_{i+1} = x_i-\gamma_i (J(x_i)^T J(x_i))^{-1} J^T(x_i) r(x_i).
\end{aligned}
\right.
\end{equation}

Cette méthode peut être vue comme une méthode de Newton incomplète, car $J(x_i)^T J(x_i)$ représente seulement le premier terme dans le calcul de la Hessienne. 

Le pas de descente $\gamma_i$ est à choisir avec un algorithme de recherche de pas. 


# Chargement des librairies

In [13]:
import numpy as np
import numpy.testing as npt

import pylab as pl
import matplotlib.pyplot as plt
#%matplotlib notebook

n = 100

# La règle de Goldstein-Price pour la recherche du pas

Les algorithmes de recherche de pas assurent que la fonction décroît de manière suffisante et que le pas n'est pas trop petit, pour accélerer la convergence. 

Soit $x$ un point et $d$ une direction de descente, i.e. $\nabla f(x) \cdot d<0$. Considérons la fonction de mérite $q(t) = f(x+td)$. On peut voir que, par définition de la direction de descente $d$, on a $q'(0) = d\cdot \nabla f(x)<0$.


Considérons $m_1<m_2 \in (0,1)$, $m_1<1/2$. Pour un pas de descente $t>0$, nous avons les trois décisions suivantes :

**(a) Acceptation du pas** :
$\displaystyle m_2 q'(0) \leq \frac{q(t)-q(0)}{t}\leq m_1 q'(0)$.

**(b) Pas trop grand** : $\displaystyle m_1 q'(0)<\frac{q(t)-q(0)}{t}$.

**(c) Pas trop petit** : $\displaystyle \frac{q(t)-q(0)}{t}< m_2q'(0)$.

Ces décisions sont placées dans une boucle et à chaque itération un intervalle $[t_l,t_r]$ contenant un pas acceptable est mis à jour. Si la condition (a) est verifiée, le pas actuel convient. 

Sinon, tant qu'une borne supérieure n'a pas été trouvée, on augmente le pas (en le multipliant par 2, par exemple). Si $t_r$ existe, alors on choisit un nouveau pas dans $[t_l,t_r]$, par exemple $t = (t_l+t_r)/2$. Si (b) est verifiée, alors $t_r=t$ (nouvelle borne supérieure). Si (c) est verifiée alors $t_l=t$ (nouvelle borne inférieure).  

Pour des raisons de simplicité les deux conditions (a) et (b) (la condition (c) étant la complémentaire) seront à implémenter en utilisant des fonctions qui prennent comme entrées les variables $t,q(0),q(t),q'(0),m_1,m_2$ notées respectivement `t,q0,qt,qp0,qpt,m1,m2` et qui renvoient `True` si la condition est vérifiée.

**Q1)** Complétez la fonction ci-dessous qui correspond à la condition (a).

In [1]:
def aGP(t,q0,qt,qp0,m1,m2):
    # définir un booléen qui renvoit True si la 
    # condition (a) est vérifiée
    
    if m2*qp0<= (qt-q0)/t <= m1*qp0:
      return True
    return False

**Q2)** Complétez la fonction ci-dessous qui correspond à la condition (b).

In [2]:
def bGP(t,q0,qt,qp0,m1,m2):
    # définir un booléen qui renvoit True si la 
    # condition (b) est verifiée
    
    if (qt-q0)/t > m1*qp0:
      return True
    return False
    

Les fonctions ci-dessous (à ne pas modifier) effectuent la recherche d'un pas qui respecte les régles de Goldstein-Price. Elles utilisent les fonctions de décision implementées dans les cellules précédentes. 

In [68]:
def GPDecision(t,q0,qt,qp0,qpt,m1,m2):
    # function which returns 
    # 1 if the step can be accepted
    # 2 if the step is too big
    # 3 if the step is too small
    if aGP(t,q0,qt,qp0,m1,m2):
        return 1
    elif bGP(t,q0,qt,qp0,m1,m2):
        return 2
    else: 
        return 3
    
# linesearch
def GP_linesearch(x0,d,fun,gradf,m1,m2):
    # initialisation 
    # 
    t = 1
    tl = 0 # lower bound
    tr = 0 # upper bound
    while (1==1):
        #print(tl," ",tr)
        q0 = fun(x0)  # q(0)
        qt = fun(x0+t*d) #q(t)
        # gradient direction
        gd = gradf(x0)
        qp0 = np.dot(gd,d) # q'(0)
        qpt = np.dot(gradf(x0+t*d),d) # q'(t)
            #print(tl," ",tr)
        dec = GPDecision(t,q0,qt,qp0,qpt,m1,m2) 
        #print("t=",t,"q0=",q0,"qt=",qt,"qp0=",qp0,"m1=",m1,"m2=",m2)
        #print(dec)   
        if dec==1:
            step=t   # we found a good step
            break
        elif dec==2:      
            tr = t  # step too big, update upper bound
        else:
            tl = t  # step too small, update lower bound
        
        # if no upper bound exists, pick a bigger t
        if(tr==0):
            t = 2*tl
        else:  
            # pick the midpoint
            t = 0.5*(tl+tr)
        
        if(abs(tr-tl))<1e-15:
            print("Something is wrong!")
            break
    return step

# La fonction de Rosenbrock

On considère pour $n \geq 2$ la fonction $f:\Bbb{R}^n \to \Bbb{R}$ définie par
$$f(x) = \sum_{i=1}^{n-1} 100(x_{i+1}-x_i^2)^2+(1-x_i)^2$$
admettant un unique minimiseur $x^* = (1,1,...,1)$ sur $\Bbb{R}^n$.

Cette fonction est souvent utilisée comme test pour des algorithmes d'optimisation. Son mauvais contitionnement fait que les algorithmes basés sur une descente de gradient ont une convergence trés lente vers le minimum.

En vue de la forme de l'algorithme de Gauss-Newton nous allons travailler avec les $2n-2$ fonctions de moindres carrés $r_i$, plutôt qu'avec la fonction $f$ :
$$ r_{i}(x) = 10(x_{i+1}-x_i^2),\ \ r_{i+(n-1)}(x) = 1-x_i, 1\leq i \leq n-1.$$
 
 
**Q3)** Complétez le code ci-dessous qui pour un vecteur $x$ donné renvoit la fonction 
$r(x)=\begin{pmatrix} r_1(x) \\ r_2(x) \\ ... \\ r_{2n-2}(x) \end{pmatrix}.$

In [20]:
def r(x):
    """ Compute the Rosenbrock function.
    Only use the variables in the function signature.
    """
    #n=len(x) en fait n=100 (enonce)
    r=[]
    for i in range(n-1):
      r.append(10*(x[i+1]-x[i]**2))
    for i in range(n-1):
      r.append(1-x[i])
    return r
    

**Q4)** Complétez le code ci-dessous qui pour un vecteur $x$ donné renvoit la matrice jacobienne associée à la fonction $r$ précédemment définie.

In [36]:
def Jac(x):
    """ Compute the Jacobian function.
    La Jacobienne contient sur chaque ligne les derivees partielles de r_i par rapport a chaque variable
    Only use the variables in the function signature.
    """
    Jac=[]
    for i in range(n-1):
      L=np.zeros(n)
      L[i]=-20*x[i]
      L[i+1]=10
      Jac.append(L)
    for i in range(n-1):
      L=np.zeros(n)
      L[i]=-1
      Jac.append(L)
    return Jac
    

A l'aide des fonctions $r_i$ et de leurs dérivées, on calcule maintenant la valeur de la fonction objectif et de son gradient.

**Q5)** Completez la fonction ci-dessous pour calculer la fonction objectif $J(x) = \sum_{i=1}^{2n-2} r_i(x)^2$.

In [9]:
# objective function
def J(x):
    """ Compute the Objective function.
    Only use the variables in the function signature.
    """
    return np.dot(r(x),r(x))
    

**Q6)** Completez la fonction ci-dessous qui calcule le gradient de la fonction objectif $\nabla J(x) = 2Dr(x)^T r(x)$ où $Dr(x)$ est la matrice Jacobienne de $r$.

In [38]:
# gradient        
def GradJ(x):
    """ Compute the Objective function.
    Only use the variables in the function signature.
    """
    Dr=Jac(x)
    rx=r(x)
    return 2*np.dot(np.transpose(Dr),rx)
    

Le but de cette question est, pour $x$ donné, de calculer la direction de descente de Gauss-Newton donnée par :
$$d(x)=-(J(x)^T J(x))^{-1} J^T(x)r(x).$$
On rappelle qu'un calcul du type $A^{-1}b$ implique la résolution du systeme linéaire $Ax=b$. La routine `np.linalg.solve` permet de résoudre un tel système linéaire. 

**Q7)** Complétez le code ci-dessous ayant pour arguments d'entrée la matrice Jacobienne $J(x)$ notée `Jr` et le vecteur $r(x)$ noté `vec_r` et qui renverra la direction de descente correspondante.

In [66]:
def GNdir(Jr,vec_r):
  A=np.dot(np.transpose(Jr),Jr)
  b=np.dot(np.transpose(Jr),vec_r)
  return -np.linalg.solve(A,b)

In [16]:
# Choix de l'initialisation et de divers paramètres

x0 = np.zeros(n)

# une initialisation "aleatoire"
for i in range(0,n):
    x0[i] = 2*(-1)**i+0.2**i

analytic = np.ones(n)

m1 = 0.1
m2 = 0.9
Tol = 1e-16
Maxiter = 300

# Choix de la direction de descente

Dans la cellule suivante on implémente le choix de la direction de descente. Pour une comparaison facile, on proposera soit la direction opposée au gradient : $d=-\nabla f(x)$ (choix classique), soit la direction proposée par l'algorithme de Gauss-Newton. La variable `v` permettra de changer facilement entre les deux choix proposés. 

In [57]:
def Choix(v,fun,gradf,x0):
    if(v==1): # descente gradient
        d = -gradf(x0)
    else:# Gauss-Newton
        d = GNdir(Jac(x0),r(x0))
    return d
 

# Descente de gradient avec un line search

La cellule suivante (à ne pas modifier) implémente l'algorithme générique de descente de gradient, avec une eventuelle possibilité de modifier la direction de descente. A l'intérieur de la boucle d'optimisation on pourra observer la présence de la routine de recherche d'un pas qui satisfait les conditions de Goldstein-Price. 

In [70]:
def GDlinesearch(fun,gradf,x0,Tol,maxiter,m1,m2,v=1):
    print("=================================================================================")
    if(v==1):
        print("Optimisation utilisant une descente de gradient avec recherche du pas Goldstein-Price")
    else:
        print("Optimisation utilisant Gauss-Newton avec recherche du pas Goldstein-Price")
    phist = []
    vhist = []
    ghist = []
    phist.append(x0)            # Create an array which holds the optimization history
 
    val = fun(x0)
    gd   = gradf(x0)
    vhist.append(val)
    ghist.append(gd)
    iter = 0

    while abs(np.linalg.norm(gd))>=Tol: 
        iter=iter+1
        # recherche du pas
        d = Choix(v,fun,gradf,x0)
        #rint(d)
        #print(np.dot(d,gd))
        #print("This is where it goes wrong")
        step = GP_linesearch(x0,d,fun,gradf,m1,m2)
        #print("issue fixed!")
            
        x0 = x0+step*(d)
        val = fun(x0)
        gd   = gradf(x0)
        if(iter%50==0):
            print("Iter: ",iter,"| Val: ",val,"| Step: ",step," Grad: ",np.linalg.norm(gd))
        phist.append(x0)
        vhist.append(val)
        ghist.append(d)
        if(iter>maxiter):
            print('Maximum number of iterations reached!')
            break
        #if(abs(val-prevval)<Tol):
            #print('Function does not decrease enough!')
            #break
        prevval = val
    if(np.linalg.norm(d)<Tol): 
        print('Algorithm converged!')
    print('')
    print('Final output:')
    print("Iter: ",iter,"| Val: ",val,"| Step: ",step," Grad: ",np.linalg.norm(gd))
    print("=================================================================================")

    return phist,vhist,ghist

# Comparaison : Gauss-Newton vs Descente de Gradient classique

Dans la cellule suivante on pourra observer l'amélioration de la vitesse de convergence obtenue en utilisant la direction donnée par l'algorithme de Gauss-Newton. Pour la fonction Rosenbrock en 2D l'algorithme de gradient, même ayant de bonnes propriétés théoriques de convergence a du mal à converger rapidement vers l'optimum. Ceci est dû au mauvais conditionnement de la fonction autour de l'optimum $(1,1)$. L'algorithme de Gauss-Newton permet d'obtenir une convergence très rapide, en quelques itérations seulement, ceci en utilisant seulement des informations sur les dérivées premières.  

Le bon comportement de l'algorithme de Gauss Newton est justifié par l'exploitation efficace de la structure de la fonction objectif (somme de carrés). En plus, pour des fonctions $r_i$ qui ne sont pas trop oscillatoires, autour d'un minimum, la direction donnée par Gauss-Newton va s'approcher de la direction de descente de l'algorithme de Newton, avec des propriétés de convergence améliorées (voir le cours). 

In [72]:
m1=0.001
m2=0.9

# Test avec descente de Gradient

pp,vv,gg = GDlinesearch(J,GradJ,x0,Tol,Maxiter,m1,m2,v=1)

print('Number of iterations: ',len(pp))
print('Final position: ',pp[-1])
print('Difference to analytical sol: ',np.linalg.norm(analytic-pp[-1]))

print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")


# Test avec Gauss-Newton
pp2,vv2,gg2 = GDlinesearch(J,GradJ,x0,Tol,Maxiter,m1,m2,v=2)

print('Number of iterations: ',len(pp2))
print('Final position: ',pp2[-1])
print('Difference to analytical sol: ',np.linalg.norm(analytic-pp2[-1]))



Optimisation utilisant une descente de gradient avec recherche du pas Goldstein-Price
Iter:  50 | Val:  0.013131034972966029 | Step:  0.0009765625  Grad:  1.1216112743702615
Iter:  100 | Val:  0.011183335030591013 | Step:  0.0009765625  Grad:  0.19030592231643162
Iter:  150 | Val:  0.010436630757118929 | Step:  0.001953125  Grad:  0.3026932017294806
Iter:  200 | Val:  0.009725878147145654 | Step:  0.0009765625  Grad:  0.19027762258938016
Iter:  250 | Val:  0.00908684578310099 | Step:  0.001953125  Grad:  0.3113227105450119
Iter:  300 | Val:  0.008471050302016104 | Step:  0.0009765625  Grad:  0.1923175730324263
Maximum number of iterations reached!

Final output:
Iter:  301 | Val:  0.008458379399140495 | Step:  0.0009765625  Grad:  0.15943164591773656
Number of iterations:  302
Final position:  [0.99999975 1.00000062 0.99999901 1.00000135 0.99999828 1.00000208
 0.99999757 1.00000278 0.99999687 1.00000347 0.9999962  1.00000412
 0.99999557 1.00000473 0.99999497 1.00000531 0.99999441 1.000